# 卷积层

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):
    '''
    计算二维互相关运算
    :param X:输入
    :param K:卷积核
    :return:卷积的结果
    '''
    h,w = K.shape
    Y = torch.zeros(X.shape[0]-h+1,X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w] * K).sum()
    return Y

In [2]:
X = torch.tensor([[0.0,1.0,2.0],[3,4,5],[6,7,8]])
K = torch.tensor([[0.0,1],[2,3]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
# 实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernal_size):
        super().__init__()
        # weight就是卷积核
        self.weight = nn.Parameter(torch.rand(kernal_size))
        self.bias = nn.parameter(torch.zeros(1))

    def forward(self,X):
        return corr2d(X,self.weight) + self.bias

In [7]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1.0,-1.0]])

In [10]:
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [11]:
# K不能检测横向
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [12]:
# 学习由X生成Y的卷积核
# 输入通道是1，输出听到是1，核大小是(1,2)
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 批量大小维度为1，批量维度为1
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

# 迭代十次
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i+1)%2 == 0:
        print(f'batch{i+1},loss{l.sum():3f}')

batch2,loss6.883203
batch4,loss1.187283
batch6,loss0.212494
batch8,loss0.041099
batch10,loss0.009127


In [13]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9928, -0.9801]])